In [7]:
# 필요한 라이브러리 불러오기
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 데이터 불러오기
df = pd.read_csv("BP_data.csv")  # 파일 경로 조정 필요

# 결측치 처리 (평균으로 대체)
df.fillna(df.mean(numeric_only=True), inplace=True)

# 타겟과 피처 설정
target = "Blood_Pressure_Abnormality"
X = df.drop(columns=["Patient_Number", target])
y = df[target]

# 훈련/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 정규화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 텐서 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

# 신경망 모델 정의
class SimpleClassifier(nn.Module):
    def __init__(self, input_dim):
        super(SimpleClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# 모델 초기화
model = SimpleClassifier(input_dim=X_train.shape[1])

# 손실 함수와 옵티마이저
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 루프
epochs = 300
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train_tensor)
    loss = criterion(output, y_train_tensor)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# 정확도 평가
model.eval()
with torch.no_grad():
    preds = model(X_test_tensor)
    predicted = (preds > 0.5).float()
    accuracy = (predicted.eq(y_test_tensor).sum() / y_test_tensor.shape[0]).item()
    print(f"Test Accuracy: {accuracy:.4f}")

from sklearn.metrics import classification_report

# 모델 예측 (평가 모드)
model.eval()
with torch.no_grad():
    preds = model(X_test_tensor)               # 예측 확률값 (0~1)
    predicted = (preds > 0.5).float().numpy()  # 0.5 기준 이진 분류
    y_true = y_test_tensor.numpy()             # 실제값

# classification_report로 성능 지표 출력
print(classification_report(y_true, predicted, digits=4))



Epoch 10/300, Loss: 0.6837
Epoch 20/300, Loss: 0.6749
Epoch 30/300, Loss: 0.6661
Epoch 40/300, Loss: 0.6569
Epoch 50/300, Loss: 0.6469
Epoch 60/300, Loss: 0.6357
Epoch 70/300, Loss: 0.6233
Epoch 80/300, Loss: 0.6097
Epoch 90/300, Loss: 0.5954
Epoch 100/300, Loss: 0.5814
Epoch 110/300, Loss: 0.5682
Epoch 120/300, Loss: 0.5561
Epoch 130/300, Loss: 0.5450
Epoch 140/300, Loss: 0.5342
Epoch 150/300, Loss: 0.5238
Epoch 160/300, Loss: 0.5136
Epoch 170/300, Loss: 0.5038
Epoch 180/300, Loss: 0.4940
Epoch 190/300, Loss: 0.4843
Epoch 200/300, Loss: 0.4749
Epoch 210/300, Loss: 0.4659
Epoch 220/300, Loss: 0.4573
Epoch 230/300, Loss: 0.4491
Epoch 240/300, Loss: 0.4414
Epoch 250/300, Loss: 0.4341
Epoch 260/300, Loss: 0.4271
Epoch 270/300, Loss: 0.4204
Epoch 280/300, Loss: 0.4141
Epoch 290/300, Loss: 0.4082
Epoch 300/300, Loss: 0.4027
Test Accuracy: 0.8575
              precision    recall  f1-score   support

         0.0     0.9109    0.8251    0.8659       223
         1.0     0.8030    0.8983    0